# Density matrix exponentiation

This is a function to simulate DME. It follows the following protocol:
![image.png](attachment:image.png)
Goals: implement the unitary $U=e^{-i\rho\theta}$ on data qubit $\sigma$ (rotate $\sigma$ by $\theta$) according to instruction given by instruction qubit $\rho$.

### Function `swap_rotation`
First, DME uses the relation (rotate $\sigma$ by small angle $\delta = \sigma/N$).
$$\begin{align}
Tr_\rho [e^{-iSWAP\delta}\sigma\otimes\rho e^{iSWAP\delta}] &= \sigma - i\delta[\rho,\sigma] +\mathcal{O}(\delta^2)\\
&= e^{-i\rho\delta}\sigma e^{i\rho\delta}+\mathcal{O}(\delta^2)
\end{align}$$


### Function `QME`
Second, DME uses either a simple reset, or QME to dynamically reset the instruction qubit.

QME apply $\mathbb{1}$, or $\pi$-rotation in the instruction state basis $R_\nu(\pi)$ with probability $p = 0.5$

Outcomes of multiple such randomized instantiations are averaged $\to$ mimic single circuit with active reset of $\rho$

### Function `DME_evolution`
Combine the above, this function execute the above circuit (without QME) or the following circuit (with QME):

![image-2.png](attachment:image-2.png)


In [ ]:
import numpy as np
from qibo.config import raise_error
from abc import abstractmethod

import qibo
from qibo import gates, models
from enum import Enum, auto

def test_qdp_memory_usage_query_circuit():
    qdp = quantum_dynamic_programming(1,10)
    qibo.set_backend("qibolab", platform="my_platform")

class QDP_memory_type(Enum):
    default = auto()
    reset = auto()
    quantum_measurement_emulation = auto()

class quantum_dynamic_programming:
    def __init__(self, num_target_qubits, num_instruction_qubits, number_muq_per_call):
        self.num_target_qubits = num_target_qubits,
        self.num_instruction_qubits = num_instruction_qubits
        #SAM: fix id on each qubit, if workin put in work_reg, otherwise instruction_reg
        self.id_target_reg = np.arrange(0,num_target_qubits,1)
        self.id_instruction_reg = np.arrange(0,num_instruction_qubits,1)
        #SAM: since instruction can be multi qubit, id_current_instruction_reg should be list/arr
        self.id_current_instruction_reg = 0
        self.M = number_muq_per_call
        self.memory_type = QDP_memory_type.default
        self.c = models.Circuit(self.num_target_qubits + self.num_instruction_qubits)
    
    def __call__(self):
        # return entire circuit
        return self.memory_call_circuit()
        # for now we assume that the recursion step make only 1 type memory call per QDP step

    def memory_call_circuit(self):
        # return the entire circuit
        if self.memory_type == QDP_memory_type.default:
            # SAM: should contain self.M * num_instruction_qubits_per_query
            for register in self.id_instruction_reg[self.current_instruction_register():self.M+self.current_instruction_register()]:
                self.memory_usage_query_circuit()
                self.trace_instruction_qubit()
                self.increment_current_instruction_register()
        
        elif self.memory_type == QDP_memory_type.reset:
            for _register_use in range(self.M):
                self.memory_usage_query_circuit()
                self.trace_instruction_qubit()
                self.single_register_reset(self.get_current_instruction_register())
                
        elif self.memory_type == QDP_memory_type.quantum_measurement_emulation:
            for _register_use in range(self.M):
                self.memory_usage_query_circuit()
                self.trace_instruction_qubit()
                self.QME(self.get_current_instruction_register())
                
        return self.c

    def single_register_reset(self,register):
        self.c.add(gates.reset(register))

    def all_register_reset(self):
        for qubit in self.num_instruction_qubits:
            self.c.add(gates.reset(qubit))
        self.id_current_instruction_reg = 0

    def circuit_reset(self):
        self.c = models.Circuit(self.num_target_qubits + self.num_instruction_qubits)
    
    def increment_current_instruction_register(self):
        self.id_current_instruction_reg += 1
      
    @abstractmethod
    def memory_usage_query_circuit(self, s, N, id_target_reg, id_instruction_reg):
        #todo: ask them how to change gates duration
        self.c.add(gates.N(id_target_reg,id_instruction_reg))
        #raise_error(NotImplementedError)
    
    def QME(self,register):
        import random
        coin_flip = random.choice([0, 1])
        if coin_flip == 0:
            QME_gate = gates.R_nu(np.pi,register) # nu is the vector parallel to rho
        elif coin_flip == 1:
            QME_gate = gates.identity(register)
        self.c.add(QME_gate)

    def trace_instruction_qubit(self):
        for qubit in self.id_instruction_reg:
            self.c.add(gates.M(qubit))

In [ ]:
def partial_trace(dm, sub_index):
    """
    Calculate the partial trace over the second qubit.

    Parameters:
    dm : numpy.ndarray
        The 4x4 density matrix of a two-qubit system.
    sub_index : int
        The index of the subsystem to trace out. 1 for tracing out the first qubit,
        and 2 for tracing out the second qubit.

    Returns:
    numpy.ndarray
        The 2x2 reduced density matrix of the specified qubit.
    """
    # Reshape dm from a 4x4 matrix into a 2x2x2x2 tensor
    reshaped_dm = dm.reshape(2, 2, 2, 2)

    # Perform the partial trace by summing out the inner 2x2 matrix
    if sub_index not in [1, 2]:
        raise ValueError('Sub index should be in range [1, 2]')

    if sub_index == 1:
        # Trace out the second subsystem
        reduced_dm = np.einsum('ijik->jk', reshaped_dm)

    if sub_index == 2:
        # Trace out the first subsystem
        reduced_dm = np.einsum('jiki->jk', reshaped_dm)

    return reduced_dm

In [ ]:
class quantum_dynamic_programming_numpy:
    # SAM: matrix representatio of QDP call (circuit)
    def __init__(self, sigma):
        self.sigma = sigma

    @abstractmethod
    def memory_usage_query(self,N_op,rho,sigma = None):
        if sigma == None:
            sigma = self.sigma
        return trace_1(np.exp(-1j*N_op) * tensor(rho,sigma) * np.exp(1j*N_op))
        
    #todo look up qibo evolution

    def memory_call(self):
        raise_error(NotImplementedError)



class density_matrix_exponentiation(quantum_dynamic_programming):
    def __init__(self, rho, sigma, theta, N):
        self.rho = rho # instruction qubit
        self.sigma = sigma # data_qubit
        self.theta = theta # overall rotation angle
        self.N = N # number of steps
        self.delta = theta/N # small rotation angle
        self.partial_swap_negative = np.exp(-1j*swap()*self.delta)
        self.partial_swap_positive = np.exp(1j*swap()*self.delta)

    def DME_evolution(self):
        current_sigma, current_rho = self.sigma, self.rho
        for _ in range(self.N):
            current_sigma, current_rho = self.swap_rotation(current_sigma, current_rho)
        return current_sigma, current_rho

    def swap_rotation(self, current_sigma, current_rho, QME = False):        
        system = tensor(current_sigma, current_rho)
        unitary_evolution = self.partial_swap_negative * system * self.partial_swap_positive
        new_sigma = trace_rho(unitary_evolution)
        if QME == True:
            new_rho = self.QME(current_rho)
        elif QME == False:
            new_rho = self.rho 
        return new_sigma, new_rho

    def QME(self, current_rho):
        import random
        coin_flip = random.choice([0, 1])
        if coin_flip == 0:
            QME_gate = R_nu(np.pi) # nu is the vector parallel to rho
        elif coin_flip == 1:
            QME_gate = identity()
        new_rho = QME_gate * current_rho
        return new_rho

    def measurement(self):
        # measurement or state tomography
        current_sigma, current_rho = self.DME_evolution()
        final_sigma = measurement_gate * current_sigma
        final_rho = measurement_gate * current_rho
        return final_sigma, final_rho